# Airflow and PostgreSQL

Airflow allows you to run SQL queries in your tasks. You can connect to virtually any database, but in this example we will connect to a PosgreSQL database. Connecting to a database is very easy in Airflow, especially because the UI has a nice way to do it.

Before you start, you have to install the PosgreSQL client library. To do so, you can run the following command:

`pip install apache-airflow-providers-postgres`

From now on, Airflow will know how to connect to a postgres database. _If you want to connect to a different database, you can do it by installing a different provider. Check out [this webpage](https://www.astronomer.io/guides/connections) to get more info_

As mentioned, the Airflow UI has a nice way to connect to a database. Open your UI, click on 'Admin' and then on 'Connections'.

<p align="center">
    <img src="images/AirFlow_SQL.png" width="500"/>
</p>

In the next window, you can select the connection you want to configure. Take a look at all possibilities Airflow offers! Let's click on the `PostgreSQL` connection. You will see the following:

<p align="center">
    <img src="images/AirFlow_SQL2.png" width="500"/>
</p>

Fill the fields with your data, but take into account that `schema` here is the name of your database and `login` is your username. Just for demonstration purposes, we will use the `Pagila` database.

You are ready to use PostgreSQL from Airflow. In the next example, we are creating a new table with animal names. You will find the same code in `dag_test_sql.py`.

In [ ]:
import datetime

from airflow import DAG
from airflow.providers.postgres.operators.postgres import PostgresOperator

with DAG(
    dag_id="test_dag_postgre",
    start_date=datetime.datetime(2020, 2, 2),
    schedule_interval="@once",
    catchup=False,
) as dag:

    create_pet_table = PostgresOperator(
        task_id="create_pet_table",
        postgres_conn_id="postgres_default",
        sql="""
            CREATE TABLE IF NOT EXISTS pet (
            pet_id SERIAL PRIMARY KEY,
            name VARCHAR NOT NULL,
            pet_type VARCHAR NOT NULL,
            birth_date DATE NOT NULL,
            OWNER VARCHAR NOT NULL);
          """,
    )

    populate_pet_table = PostgresOperator(
        task_id="populate_pet_table",
        postgres_conn_id="postgres_default",
        sql="""
            INSERT INTO pet VALUES ( 1, 'Max', 'Dog', '2018-07-05', 'Jane');
            INSERT INTO pet VALUES ( 2, 'Susie', 'Cat', '2019-05-01', 'Phil');
            INSERT INTO pet VALUES ( 3, 'Lester', 'Hamster', '2020-06-23', 'Lily');
            INSERT INTO pet VALUES ( 4, 'Quincy', 'Parrot', '2013-08-11', 'Anne');
            """,
    )

    get_all_pets = PostgresOperator(
        task_id="get_all_pets", postgres_conn_id="postgres_default", sql="SELECT * FROM pet;"
    )

    get_birth_date = PostgresOperator(
        task_id="get_birth_date",
        postgres_conn_id="postgres_default",
        sql="""
            SELECT * FROM pet
            WHERE birth_date
            BETWEEN {{ params.begin_date }} AND {{ params.end_date }};
            """,
        params={'begin_date': '2020-01-01', 'end_date': '2020-12-31'},
    )


    create_pet_table >> populate_pet_table >> get_all_pets >> get_birth_date

Defining the tasks is very similar to defining PythonOperators. The only difference is that you have to include the `postgres_conn_id` in the task definition. Then, the `sql` argument will contain the query, and the params argument will contain the variables you might want to add to the query.

If you copy that code to the `dags` folder, you can test it by running `airflow dags test dag_test_sql`. Then, in your pgAdmin, you can see the table `pets`:

<p align="center">
    <img src="images/AirFlow_SQL3.png" width="500"/>
</p>

Same way you connected to your localhost, you can also connect to your AWS RDS, we are slowly integrating everything from Python!

# Final Note

Everything we have done so far can be also done using the command line in an easy manner as well. For example, for changing the configuration of a connection you can run the following command:

`airflow connections add <connection_name> \ --conn-uri "conn-type://<user>:<password>@<host>:<port>/<database>`

Or another example you can do in the command line is to create a new variable:

`airflow variables -s <var_name> <var_value>`

Check out the official documentation for more information. 

With this in mind, you can now start creating your own Airflow DAGs in your EC2 instances without having to worry about the Airflow UI. Just leave it running as shown in the `Cloud Basics` module and you will have a schedule running nonstop!

# Summary

- The configuration of Airflow in the UI can be set to connect to different databases such as Postgresql, or even AWS RDS.